In [1]:
#make imports

import os
import sys
import pandas as pd
import numpy as np
import gzip
import json

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [3]:
#load main data

df_main = getDF('Electronics_5.json.gz')

In [4]:
#load meta data

df_meta = getDF('meta_Electronics.json.gz')

In [5]:
#drop na

df_main.dropna(inplace=True)
df_meta.dropna(inplace=True)

In [6]:
#display columns

print(df_main.columns)
print(df_meta.columns)

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')
Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')


In [7]:
#get rid of duplicates

hashable_cols_main = []
for column in df_main.columns:
    if str(df_main[column].dtype) != 'object':
        hashable_cols_main.append(column)

hashable_cols_meta = []
for column in df_meta.columns:
    if str(df_meta[column].dtype) != 'object':
        hashable_cols_meta.append(column)

if (len(hashable_cols_main) > 0):
    df_main.drop_duplicates(subset=hashable_cols_main, inplace=True)

if (len(hashable_cols_meta) > 0):
    df_meta.drop_duplicates(subset=hashable_cols_meta, inplace=True)

In [8]:
#make df for headphones metadata after filtering for matches in title
headphone_meta_df = df_meta[df_meta['title'].str.contains('headphone|headphones', case=False)]

asins = set(headphone_meta_df['asin'])

#make df for headphones review after filtering for matches using asin
headphone_review_df = df_main[df_main['asin'].isin(asins)]

In [ ]:
#dump to pickle
import pickle

headphone_meta_df.to_pickle('headphone_meta_df.pkl')
headphone_review_df.to_pickle('headphone_review_df.pkl')